# Advent of Code

This notebook contains my solutions for the 2020 version of [Advent of Code](https://adventofcode.com/2020).

### Imports and Dataimport

In [1]:
from itertools   import count
from typing      import Tuple, List

import re

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    with open(f'data/day{day}.txt') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))

## Day 1: Report Repair

### Part 1
The first challenge is to find two numbers in a list that add up to 2020 and return their product.

In [17]:
test1_input = [1721, 979, 366, 299, 675, 1456]
# 1721 + 299 = 2020 and 1721 * 299 = 514579
test1_1output = 514579

def twosum(nums):
    complements = set()
    for x in nums:
        y = 2020 - x
        
        if x in complements:
            return x * y
        
        complements.add(y)
        
assert twosum(set(test1_input)) == test1_1output

input1 = set(data(1, int))
twosum(input1)

926464

### Part 2
The second part of the puzzle is to find the three distinct numbers that add to 2020 and return their product.

In [18]:
test1_input = [1721, 979, 366, 299, 675, 1456]
# 979 + 366 + 675 = 2020 and 979 * 366 * 675 = 241861950
test1_2output = 241861950

def threesum(nums):
    for x in nums:
        complements = set()
        yz = 2020 - x

        for y in nums:
            z = yz - y

            if y in complements:
                return x * y * z

            complements.add(z)

assert threesum(set(test1_input)) == test1_2output

input1 = set(data(1, int))
threesum(input1)

65656536

## Day 2: Password Philosophy

### Part 1
The First part of this days challenge is to count how many passwords are valid according to their policies.

A given password string is considered valid if it contains between min and max instances of a specific character, where min, max and the character are specified in the policy.

In [5]:
test2_input1 = '1-3 a: abcde'
test2_input2 = '1-3 b: cdefg'
test2_input3 = '2-9 c: ccccccccc'
test2_1output1 = True
test2_1output2 = False
test2_1output3 = True

Pw_Policy = Tuple[int, int, str, str]

def parse_pw_policy(line: str) -> Pw_Policy:
    "Given '1-3 b: cdefg', return (1, 3, 'b', 'cdefg')."
    mmin, mmax, letter, pw = re.findall(r'[^-:\s]+', line)
    return (int(mmin), int(mmax), letter, pw)

def check_pw(policy) -> bool:
    mmin, mmax, letter, pw = policy
    return mmin <= pw.count(letter) <= mmax

assert check_pw(parse_pw_policy(test2_input1)) == test2_1output1
assert check_pw(parse_pw_policy(test2_input2)) == test2_1output2
assert check_pw(parse_pw_policy(test2_input3)) == test2_1output3

input2: List[Tuple] = data(2, parse_pw_policy)

sum(map(check_pw, input2))

378

### Part 2
The second part of this puzzle changes the interpretation of the password policy.
This digits that used to express min and max before now denote a position in the password string, however the index starts at 1.

A password is considered valid with this new policy if it has the specified letter at exactly one of the two given positions. How many passwords are valid?

In [6]:
test2_2output1 = True
test2_2output2 = False
test2_2output3 = False

def check_pw2(policy, offset=1) -> bool:
    first, second, letter, pw = policy
    return (pw[first - offset] == letter) ^ (pw[second - offset] == letter)

assert check_pw2(parse_pw_policy(test2_input1)) == test2_2output1
assert check_pw2(parse_pw_policy(test2_input2)) == test2_2output2
assert check_pw2(parse_pw_policy(test2_input3)) == test2_2output3

sum(map(check_pw2, input2))

280

## Day 3: Toboggan Trajectory

### Part 1
This challenge provides a map that contains '.' for free spaces and '#' for trees.

The questions is how many trees are encountered for a given map with a path that takes 3 steps right and 1 down until the bottom of the map is reached with the assumption that the pattern specified in the map repeats infinitely to the right.

In [7]:
test3_input = ['..##.......',
               '#...#...#..',
               '.#....#..#.',
               '..#.#...#.#',
               '.#...##..#.',
               '..#.##.....',
               '.#.#.#....#',
               '.#........#',
               '#.##...#...',
               '#...##....#',
               '.#..#...#.#']
test3_1output = 7

Grid = List[str]

def count_trees_along_slope(grid, dx=3, dy=1, tree='#'):
    height = len(grid)
    width = len(grid[0])
    return sum((grid[row][col % width] == tree)
               for row, col in zip(range(0, height, dy), count(0, dx)))

assert count_trees_along_slope(test3_input, dx=3, dy=1, tree='#') == test3_1output

input3 : Grid = data(3)

count_trees_along_slope(input3)

220

### Part2
The second challenge for this day asks for the product of the encountered trees for paths with different slopes.

In [8]:
test3_2output = 336

def count_trees_along_all_slopes(grid):
    def t(dx, dy):
        return count_trees_along_slope(grid, dx, dy)
    return t(1, 1) * t(3, 1) * t(5, 1) * t(7, 1) * t(1, 2)

assert count_trees_along_all_slopes(test3_input) == test3_2output

count_trees_along_all_slopes(input3)

2138320800

## Day 4: Passport Processing

### Part 1
For this challenge a batch of passports is given. Each passport is represented as a sequence of key:value pairs. 
A passport is valid if it contains all expected fields while country id is optional. How many passports are valid?

In [9]:
test4_input1 = "ecl:gry pid:860033327 eyr:2020 hcl:#fffffd byr:1937 iyr:2017 cid:147 hgt:183cm"
test4_input2 = "iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884 hcl:#cfa07d byr:1929"
test4_input3 = "hcl:#ae17e1 iyr:2013 eyr:2024 ecl:brn pid:760753108 byr:1931 hgt:179cm"
test4_input4 = "hcl:#cfa07d eyr:2025 pid:166559648 iyr:2011 ecl:brn hgt:59in"

test4_output1 = True
test4_output2 = False
test4_output3 = True
test4_output4 = False

Passport = dict

def parse_passport(passport_string: str) -> Passport:
    return Passport(re.findall(r'([a-z]+):([^\s]+)', passport_string))

required_fields = {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'}

def check_passport_fields(Passport) -> bool:
    return required_fields.issubset(Passport.keys())

assert check_passport_fields(parse_passport(test4_input1)) == test4_output1
assert check_passport_fields(parse_passport(test4_input2)) == test4_output2
assert check_passport_fields(parse_passport(test4_input3)) == test4_output3
assert check_passport_fields(parse_passport(test4_input4)) == test4_output4

input4 : List[Passport] = data(4, parse_passport, '\n\n')

sum(map(check_passport_fields, input4))

254

### Part 2
The second part of the puzzle requires passports to not only posses all required fields but also that the values of these fields obey specific rules. How many passports are valid according to the new stricter rules?

In [10]:
test4_2input1 = "eyr:1972 cid:100 hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926"
test4_2input2 = "iyr:2019 hcl:#602927 eyr:1967 hgt:170cm ecl:grn pid:012533040 byr:1946"
test4_2input3 = "hcl:dab227 iyr:2012 ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277"
test4_2input4 = "hgt:59cm ecl:zzz eyr:2038 hcl:74454a iyr:2023 pid:3556412378 byr:2007"

test4_2output1 = False
test4_2output2 = False
test4_2output3 = False
test4_2output4 = False

test4_2input5 = "pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980 hcl:#623a2f"
test4_2input6 = "eyr:2029 ecl:blu cid:129 byr:1989 iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm"
test4_2input7 = "hcl:#888785 hgt:164cm byr:2001 iyr:2015 cid:88 pid:545766238 ecl:hzl eyr:2022"
test4_2input8 = "iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719"

test4_2output5 = True
test4_2output6 = True
test4_2output7 = True
test4_2output8 = True

def check_passport_values(Passport) -> bool:
    '''Passport fields are considered valid acording to these rules:
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.'''
    
    byr : bool = 1920 <= int(Passport['byr']) <= 2002
    iyr : bool = 2010 <= int(Passport['iyr']) <= 2020
    eyr : bool = 2020 <= int(Passport['eyr']) <= 2030
    hgt : bool = (((Passport['hgt'][-2:]=='cm') and (150 <= int(Passport['hgt'][:-2]) <= 193)) or
                  ((Passport['hgt'][-2:]=='in') and (59 <= int(Passport['hgt'][:-2]) <= 76)))
    hcl : bool = bool(re.match('#[0-9a-f]{6}$', Passport['hcl']))
    eyecolors = {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}
    ecl : bool = Passport['ecl'] in eyecolors
    pid : bool = bool(re.match('[0-9]{9}$', Passport['pid']))
    
    return all([byr, iyr, eyr, hgt, hcl, ecl, pid])

assert check_passport_values(parse_passport(test4_2input1)) == test4_2output1
assert check_passport_values(parse_passport(test4_2input2)) == test4_2output2
assert check_passport_values(parse_passport(test4_2input3)) == test4_2output3
assert check_passport_values(parse_passport(test4_2input4)) == test4_2output4
assert check_passport_values(parse_passport(test4_2input5)) == test4_2output5
assert check_passport_values(parse_passport(test4_2input6)) == test4_2output6
assert check_passport_values(parse_passport(test4_2input7)) == test4_2output7
assert check_passport_values(parse_passport(test4_2input8)) == test4_2output8

sum(check_passport_values(passport)
    for passport in input4 if check_passport_fields(passport))

184

## Day 5: Binary Boarding

### Part 1
This puzzle provides a list of boarding passes that each specify one seat in the airplane. The airlane uses a binary space partioning approach for seating. Each boarding pass contains 10 characters, the first 7 are either 'F' or 'B' and specify the exact row. The last 3 characters can either be 'R' or 'L' and specify the column of the seat.

The airplane has 128 rows (0-127) and 8 columns(0-7). 'F' means front and indicates the lower half of the search space, 'B' means back and designates the upper half of the respective interval. For the column the convention is that 'L' can be interpreted as left and is meant to designate the lower half of the search space, while 'R' or right means the upper half.

Additionaly each seat in the airplane has a seat ID that is calculated by row * 8 + column.

The challenge is to find the highest seat ID of all provided boarding passes.

In [11]:
test5_input1 = 'FBFBBFFRLR'
test5_input2 = 'BFFFBBFRRR'
test5_input3 = 'FFFBBBFRRR'
test5_input4 = 'BBFFBBFRLL'

test5_output1 = 357
test5_output2 = 567
test5_output3 = 119
test5_output4 = 820

Boarding_Pass = str

def parse_boarding_pass(Boarding_Pass) -> str:
    "translate the string to binary according to the specified conventions for rows and columns"
    table = str.maketrans('FBRL', '0110')
    return Boarding_Pass.translate(table)

def get_seat_ID(binary_boarding_pass) -> int:
    #the outcommented code was writen before realizing that
    #first 7 digits * 8 (equivalent to << 3) + last 3 digits is the same as original string in binary 😅
    #row = int(binary_boarding_pass[:-3], 2)
    #column = int(binary_boarding_pass[-3:], 2)
    #return row * 8 + column
    return int(binary_boarding_pass, 2)

assert get_seat_ID(parse_boarding_pass(test5_input1)) == test5_output1
assert get_seat_ID(parse_boarding_pass(test5_input2)) == test5_output2
assert get_seat_ID(parse_boarding_pass(test5_input3)) == test5_output3
assert get_seat_ID(parse_boarding_pass(test5_input4)) == test5_output4

input5 = data(5, parser=parse_boarding_pass)
max(map(get_seat_ID, input5))

955

### Part 2
For the second part of this days puzzle a specific seat ID has to be found.
The wanted seat ID is not in the list of provided boarding passes.
This is not unambigously however so additionaly it is stated that the ID is not at the very front or back of the airplane but within the bulk of all the other seat IDs, as the adjacent IDs are also in the provided list.

In [12]:
seat_IDs = set(map(get_seat_ID, input5))

def find_seat_ID():
    id = (set(range(min(seat_IDs), max(seat_IDs))) - (seat_IDs)).pop()
    return id

find_seat_ID()

569

## Day 6: Custom Customs

### Part1
This puzzle provides a list with answers of groups for customs declarations questions. Groups are separted by blank lines. A group can contain answers for multiple persons, all answers for one person are in a single line. There are 26 different custom questions and a positive answer is represented by the corresponding letter in the list. The challenge is to count all different questions that are group answered with yes and to sum up these counts for all groups.

In [13]:
test6_input1 = '''abc

a
b
c

ab
ac

a
a
a
a

b'''
test6_output1 = 11

Group = List[str]

def parse_group(group_str: str) -> Group:
    return group_str.splitlines()

def count_groups(groups: List[Group]) -> int:
    return sum(len(set.union(*map(set, group))) for group in groups)

assert count_groups([parse_group(group) for group in test6_input1.split('\n\n')]) == test6_output1

input6: List[Group] = data(6, parse_group, sep='\n\n')
    
count_groups(input6)

6596

### Part 2
For the second part the task is modified slightly to only count the answers in a group that were answered with yes by every person in the group and to then count these for all groups.

In [14]:
test6_2output1 = 6

def count_groups2(groups: List[Group]) -> int:
    return sum(len(set.intersection(*map(set, group))) for group in groups)

assert count_groups2([parse_group(group) for group in test6_input1.split('\n\n')]) == test6_2output1

count_groups2(input6)

3219

## Day 7: Handy Haversacks

In [15]:
test7_input1 = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.'''

test7_output1 = 4